In [1]:
import pandas as pd
import numpy as np

In [2]:
base_line = pd.read_json('sample_submission.json')
base_line

,plan_id,image,goal_eng,task_type,plan
0,0,2176356811276008251_0.png,Put the gray cat in an orange container.,0,"[[fill_this_place, [fill_this_place, fill_this..."
1,1,2176356811276008251_0.png,Put the toy on the floor.,0,"[[fill_this_place, [fill_this_place, fill_this..."
2,2,2556860223905553708_0.png,Take the garlic from the table and put it on a...,0,"[[fill_this_place, [fill_this_place, fill_this..."
3,3,2556860223905553708_0.png,Pick up the apple from the floor.,0,"[[fill_this_place, [fill_this_place, fill_this..."
4,4,2657671826402908683_0.png,Put the kitten on the table.,0,"[[fill_this_place, [fill_this_place, fill_this..."
...,...,...,...,...,...
65,65,6910633416171335463_1.png,Put the corn on the chair.,0,"[[fill_this_place, [fill_this_place, fill_this..."
66,66,7653764438352825554_0.png,Put the cucumber in an orange container.,0,"[[fill_this_place, [fill_this_place, fill_this..."
67,67,7653764438352825554_0.png,Take the cucumber to the green container.,0,"[[fill_this_place, [fill_this_place, fill_this..."
68,68,7808152141612422513_0.png,Put the cucumber in a green container.,0,"[[fill_this_place, [fill_this_place, fill_this..."


In [99]:
ss = pd.read_json(r'C:\Users\ivox\Documents\keysMTFI\разархив\МФТИ\МФТИ\train_dataset\train_dataset.json')
ss

,plan_id,image,goal_eng,task_type,plan
0,0,-9537287721930320_0.png,Put the orange kitten in the green box.,0,"[[move_to, [unspecified, orange kitten]], [pic..."
1,1,-9537287721930320_0.png,Take a toy cube from the floor and put it on t...,0,"[[move_to, [floor, toy cube]], [pick_up, [floo..."
2,2,-31883670212970234_0.png,Move the toy cat from the table to the nightst...,0,"[[move_to, [table, toy cat]], [pick_up, [table..."
3,3,-31883670212970234_0.png,Put the cucumber in the drawer.,0,"[[move_to, [unspecified, cucumber]], [pick_up,..."
4,4,-167436868961384133_0.png,Take a cucumber from the table and put it in a...,0,"[[move_to, [table, cucumber]], [pick_up, [tabl..."
...,...,...,...,...,...
225,225,1713598062215361162_0.png,Pick up a tomato from the floor and put it on ...,0,"[[move_to, [floor, tomato]], [pick_up, [floor,..."
226,226,1808600344675605482_0.png,Take the cube from the table.,0,"[[pick_up, [table, cube]]]"
227,227,1808600344675605482_0.png,Drive up to the bedside table.,0,"[[move_to, [bedside table, unspecified]]]"
228,228,2053765613034393610_0.png,Put the toy cat on the table.,0,"[[put, [table, toy cat]]]"


In [100]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

# Предобработанные данные (предполагается, что у вас уже есть токенизированные и подготовленные данные)
# input_texts = ["Put the orange kitten in the green box."]
input_texts = ss['goal_eng']
output_texts = ss['plan']
# output_texts = [['move_to', ['unspecified', 'orange kitten']],
#                 ['pick_up', ['unspecified', 'orange kitten']],
#                 ['move_to', ['green box', 'orange kitten']],
#                 ['put', ['green box', 'orange kitten']]]

# Создание словарей для слов и тегов
input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts)
input_vocab_size = len(input_tokenizer.word_index) + 1

output_tokenizer = Tokenizer()
output_tokenizer.fit_on_texts([word for sublist in output_texts for word in sublist])
output_vocab_size = len(output_tokenizer.word_index) + 1

# Подготовка входных данных (sequences - последовательности чисел)
input_sequences = input_tokenizer.texts_to_sequences(input_texts)
max_input_length = max([len(sequence) for sequence in input_sequences])
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_input_length, padding='post')

# Подготовка выходных данных
output_sequences = []
for seq in output_texts:
    output_sequence = []
    for word in seq:
        output_sequence.append(output_tokenizer.word_index[word])
    output_sequences.append(output_sequence)

# Преобразование выходных данных в формат one-hot encoding
max_output_length = max([len(sequence) for sequence in output_sequences])
padded_output_sequences = pad_sequences(output_sequences, maxlen=max_output_length, padding='post')
padded_output_sequences = np.array([np.eye(output_vocab_size)[sequence] for sequence in padded_output_sequences])

# Создание модели нейронной сети
model = Sequential()
model.add(Embedding(input_vocab_size, 128, input_length=max_input_length, mask_zero=True))
model.add(LSTM(128))
model.add(Dense(output_vocab_size, activation='softmax'))

# Компиляция модели
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Обучение модели
model.fit(padded_input_sequences, padded_output_sequences, epochs=100, verbose=2)

# Пример использования модели для генерации текста
input_text = "Put the orange kitten in the green box."
input_seq = input_tokenizer.texts_to_sequences([input_text])
padded_input_seq = pad_sequences(input_seq, maxlen=max_input_length, padding='post')

predicted_output_seq = model.predict(padded_input_seq)
predicted_output_seq = np.argmax(predicted_output_seq, axis=-1)

# Преобразование обратно в текст
predicted_output_text = []
for index in predicted_output_seq[0]:
    for word, word_index in output_tokenizer.word_index.items():
        if word_index == index:
            predicted_output_text.append(word)
            break

print(predicted_output_text)


ImportError: cannot import name 'pad_sequences' from 'keras.preprocessing.sequence' (C:\Users\ivox\anaconda3\envs\tensorflow\lib\site-packages\keras\preprocessing\sequence.py)

In [3]:
df = pd.read_csv('data.csv')
df

,clear_text,task
0,Move to the living room,0
1,Go to the kitchen area,0
2,Navigate to the bedroom,0
3,Move to the dining room,0
4,Head to the office space,0
5,Go to the bathroom,0
6,Put the book on the shelf,1
7,Place the dishes in the cupboard,1
8,Put the shoes in the closet,1
9,Store the toys in the toy box,1


In [4]:
text = list(df['clear_text'])

In [51]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


lem_split_text = []
lemmatizer = WordNetLemmatizer()

for sentence in text:
    re.sub("[^a-zA-Z]"," ",sentence)
    words = nltk.word_tokenize(sentence)  # Разбиваем предложение на слова
    lem_words = [lemmatizer.lemmatize(word) for word in words if word.lower() not in set(stopwords.words("english"))]
    lem_sentence = " ".join(lem_words)  # Объединяем слова обратно в предложение
    lem_split_text.append(lem_sentence)

print(lem_split_text)

['Move living room', 'Go kitchen area', 'Navigate bedroom', 'Move dining room', 'Head office space', 'Go bathroom', 'Put book shelf', 'Place dish cupboard', 'Put shoe closet', 'Store toy toy box', 'Place grocery kitchen table', 'Put key key holder', 'Pick clothes floor', 'Grab toy put basket', 'Pick trash throw bin', 'Lift bag carry kitchen', 'Pick fallen fruit table', 'Grab newspaper recycle']


In [55]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
matrix = count.fit_transform(lem_split_text).toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 

In [95]:
X = matrix
y = df["task"].values

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)


In [96]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3).fit(x_train, y_train)


In [97]:
knn.score(x_test, y_test)

0.6666666666666666